# 02: Line Segment Intersection

*Authors: Jan Erik Swiadek, Prof. Dr. Kevin Buchin*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup  
3. Algorithms  
    3.1. Brute Force  
    3.2. Plane Sweep  
4. References  

## 1. Introduction

The **line segment intersection problem** was stated in the lecture as follows:
Given a set of line segments in the plane, the goal is to compute all points in the intersection of two or more line segments along with these respective segments.
Only **closed line segments** are considered here, i.e. each segment contains two distinct bounding endpoints.
The endpoints are therefore potential intersection points just like any other point contained in the segments.

See the following image for an example.
The input line segments are coloured <font color='orange'>orange</font> and their intersection points are marked in <font color='blue'>blue</font>.

<img style='float: left;' src='./images/02-image00.png'>

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [1]:
# Python standard library imports
import math
from typing import Any, Optional
from itertools import combinations

# Data structure, geometry and visualisation module imports
from modules.data_structures import BinaryTree, BinaryTreeDict, Comparator, ComparisonResult as CR
from modules.geometry import Point, LineSegment, PointSequenceDict, Orientation as ORT, EPSILON
from modules.visualisation import VisualisationTool, LineSegmentSetInstance, PointsMode, SweepLineMode

Additionally, we create an object for our visualisation tool and register a few example instances.

In [2]:
visualisation_tool = VisualisationTool(400, 400, LineSegmentSetInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

c = 0.5 * canvas_size
r = 0.75 * c

def circle_point(angle: float) -> Point:
    return Point(c + r * math.cos(angle), c + r * math.sin(angle))

star_vertices = [circle_point(i * 2 * math.pi / 5 + math.pi / 2) for i in range(0, 5)]
star_instance = set(LineSegment(star_vertices[i], star_vertices[(i + 2) % 5]) for i in range(0, 5))
visualisation_tool.register_example_instance("Star", star_instance)

s = 0.1 * c
t = 1.9 * c
u = (t - s) / 6

grid_instance = set((
    *(LineSegment(Point(s + i * u, s), Point(s + i * u, t)) for i in range(0, 7)),
    *(LineSegment(Point(s, s + i * u), Point(t, s + i * u)) for i in range(0, 7)),
    *(LineSegment(Point(s, s + i * u), Point(t - i * u, t)) for i in range(1, 6)),
    LineSegment(Point(s, s), Point(t, t)),
    *(LineSegment(Point(s + i * u, s), Point(t, t - i * u)) for i in range(1, 6)),
    *(LineSegment(Point(t, s + i * u), Point(s + i * u, t)) for i in range(1, 6)),
    LineSegment(Point(t, s), Point(s, t)),
    *(LineSegment(Point(t - i * u, s), Point(s, t - i * u)) for i in range(1, 6))
))
visualisation_tool.register_example_instance("Grid", grid_instance)

overlap_instance = set((
    LineSegment(Point(0.25 * c, c), Point(1.45 * c, c)),
    LineSegment(Point(0.55 * c, c), Point(1.75 * c, c)),
    LineSegment(Point(0.85 * c, c), Point(1.15 * c, c)),
    LineSegment(Point(0.25 * c, c), Point(1.75 * c, c)),
    LineSegment(Point(c, 0.25 * c), Point(c, 1.75 * c)),
    LineSegment(Point(c, 0.625 * c), Point(c, 1.375 * c)),
    LineSegment(Point(0.375 * c, 1.75 * c), Point(0.7 * c, 0.775 * c)),
    LineSegment(Point(0.75 * c, 0.625 * c), Point(0.5 * c, 1.375 * c)),
    LineSegment(Point(1.125 * c, 0.25 * c), Point(1.275 * c, 0.7 * c)),
    LineSegment(Point(1.225 * c, 0.55 * c), Point(1.5 * c, 1.375 * c))
))
visualisation_tool.register_example_instance("Overlap", overlap_instance)

## 3. Algorithms

The lecture presented two algorithms for the given problem:
A simple Brute Force algorithm and a more sophisticated Plane Sweep method.

### 3.1. Brute Force

The easiest way to solve the line segment intersection problem is to test each pair of line segments in the input for their intersection.
There are three possibilities for the intersection of two line segments:
They don't intersect, in which case nothing needs to be done, or they intersect in exactly one point, which is then added to the output together with the segments, or they're **overlapping line segments** and their intersection is again a line segment.
As we would technically need to report infinitely many intersection points in the third case, it's considered degenerate and we choose to report just the endpoints.
The used intersection computation is further explained in [notebook no. 00](./00-Basics.ipynb).

In [3]:
def brute_force_lsi(segments: set[LineSegment]) -> PointSequenceDict:
    intersections = PointSequenceDict()
    for segment1, segment2 in combinations(segments, 2):
        intersection = segment1.intersection(segment2)
        if isinstance(intersection, Point):
            intersections.add(intersection, (segment1, segment2))
        elif isinstance(intersection, LineSegment):
            intersections.add(intersection.upper, (segment1, segment2))
            intersections.add(intersection.lower, (segment1, segment2))

    return intersections

The asymptotic running time complexity of Brute Force is obviously in $\Theta(n^2)$, where $n$ denotes the number of line segments.
This is worst-case optimal, the worst case being that all pairs of line segments intersect in a distinct point.
However, the algorithm performs many "unnecessary" tests if the input line segments have few intersection points.

We now register Brute Force for visualisation.

In [4]:
visualisation_tool.register_algorithm("Brute Force", brute_force_lsi, PointsMode())

If you haven't used our interactive visualisation tool before, see [notebook no. 00](./00-Basics.ipynb) for an explanation.
The instance size is not equal to the number of points here, because an input instance consists of line segments that are each comprised of two points.
Creating $m$ points by clicking on the canvas or using the *Random* button will thus result in $\lfloor m / 2 \rfloor$ line segments, where every (non-overlapping) pair of added points forms a line segment and is displayed as such.

In [5]:
visualisation_tool.display()

__*Takeaways:*__

* The simplest algorithm can already be worst-case optimal.

* In practice, the worst case isn't the only case to consider.

### 3.2. Plane Sweep

Considering that for most practical puporses only few intersections occur in relation to the number of input segments, an output-sensitive algorithm that depends on the number of intersections is of interest.
We've already seen examples for such algorithms: Gift Wrapping and Chan's Hull for the convex hull problem.
The output-sensitive algorithm from the line segment intersection lecture uses the **plane sweep technique**, which is a general technique suitable to solving various problems in computational geometry.
It simulates a **sweep line** to pass through the input elements while maintaining the status at the current sweep line position and the event points that are yet to be visited by the sweep line.
The versatility of this technique partly comes from the fact that new event points can be computed on the fly during the course of the sweep.

Here, the **status structure** consists of all line segments currently intersected by the sweep line, while the **event queue** contains line segment endpoints and previously computed intersection points.
Like suggested in the lecture, we implement both as a balanced binary tree.
For this purpose, we define an order on each set of possible tree keys through a *comparator class*.
The used binary tree implementation is further explained in [notebook no. 00](./00-Basics.ipynb).

The keys of the event queue are event points, and items to be compared with them are also always points.
Since we want the sweep line to go from top to bottom, we prioritise points with a greater $y$-coordinate.
Such points should thus come first in the event queue order (see \[1, p. 24\]).
Moreover, we deal with the degenerate case of event points having the same $y$-coordinate by prioritising smaller $x$-coordinates, i.e. the corresponding events are handled from left to right.
Note that, in order to somewhat compensate for the inaccuracy of floating-point arithmetic, we use comparisons involving a small epsilon value throughout this notebook.
More on that can be found in [notebook no. 00](./00-Basics.ipynb).

In [6]:
class EventQueueComparator(Comparator[Point]):
    def compare(self, item: Any, key: Point) -> CR:
        if not isinstance(item, Point):
            raise TypeError("Only points can be compared with event points.")
        elif abs(item.x - key.x) <= EPSILON and abs(item.y - key.y) <= EPSILON:
            return CR.MATCH
        elif item.y - key.y > EPSILON or (key.y - item.y <= EPSILON and item.x < key.x):
            return CR.BEFORE
        else:
            return CR.AFTER

The comparator class for the status structure is less straightforward.
Because the left-to-right order of stored line segments depends on the current height of the sweep line and can change at event points, a **dynamic comparator** is required (see \[2, pp. 24–25\]).
The keys of the status stucture are line segments, which are compared to other line segments at the current sweep line height for tree insertion and deletion.
Additionally, the node keys also need to be comparable with points in order to search for all segments containing an event point and to determine its neighbouring segments.
Therefore, the comparator has to support points as items too.

Comparing a point to a line segment is easy using an orientation test (see [notebook no. 00](./00-Basics.ipynb)).
The comparison of two line segments can then be reduced to finding the intersection point of one line segment with the current sweep line and comparing this point to the other segment.
If this point is contained in the other segment as well, the location of the event point becomes relevant.
That's also true for horizontal line segments, which constitute a degenerate case and are dealt with separately.
In the other degenerate case we've mentioned before, two overlapping line segments, the order doesn't actually matter as long as it's consistent, so we simply use their coordinates tuples.

In [7]:
class StatusStructureComparator(Comparator[LineSegment]):
    def __init__(self):
        self._event_point: Optional[Point] = None

    def set_event_point(self, event_point: Point):
        self._event_point = event_point

    def compare(self, item: Any, key: LineSegment) -> CR:
        if isinstance(item, Point):
            return self._compare_point_with_segment(item, key)
        elif isinstance(item, LineSegment):
            return self._compare_segment_with_segment(item, key)
        else:
            raise TypeError("Only line segments and points can be compared with status line segments.")

    def _compare_point_with_segment(self, point: Point, segment: LineSegment) -> CR:
        if point.y - segment.upper.y > EPSILON or segment.lower.y - point.y > EPSILON:
            raise ValueError(f"Point {point} isn't in y-range of compared segment {segment}.")
        elif abs(segment.upper.y - segment.lower.y) <= EPSILON:
            if segment.upper.x - point.x > EPSILON:
                return CR.BEFORE
            elif point.x - segment.lower.x > EPSILON:
                return CR.AFTER
            else:
                return CR.MATCH
        else:
            ort = point.orientation(segment.lower, segment.upper)
            if ort is ORT.LEFT:
                return CR.BEFORE
            elif ort is ORT.RIGHT:
                return CR.AFTER
            else:
                return CR.MATCH

    def _compare_segment_with_segment(self, segment1: LineSegment, segment2: LineSegment) -> CR:
        special_cr = self._check_special_cases(segment1, segment2)
        if special_cr is not None:
            return special_cr

        x_coordinates = (segment1.upper.x, segment1.lower.x, segment2.upper.x, segment2.lower.x)
        left_point = Point(min(x_coordinates) - 1.0, self._event_point.y)
        right_point = Point(max(x_coordinates) + 1.0, self._event_point.y)
        segment1_point = segment1.intersection(LineSegment(left_point, right_point))
        segment1_point_cr = self._compare_point_with_segment(segment1_point, segment2)
        if segment1_point_cr is not CR.MATCH:
            # Check other direction as well to ensure consistency and increase robustness.
            segment2_point = segment2.intersection(LineSegment(left_point, right_point))
            if self._compare_point_with_segment(segment2_point, segment1) is not CR.MATCH:
                return segment1_point_cr

        event_point_cr = self._compare_point_with_segment(self._event_point, segment1)
        segment1_endpoint = segment1.upper if event_point_cr is CR.BEFORE else segment1.lower
        if segment1_endpoint.orientation(segment2.lower, segment2.upper) is ORT.LEFT:
            return CR.BEFORE
        else:
            return CR.AFTER

    def _check_special_cases(self, segment1: LineSegment, segment2: LineSegment) -> Optional[CR]:
        if self._event_point is None:
            raise RuntimeError("Event point has to be set for line segment comparison")
        elif self._event_point.y - segment1.upper.y > EPSILON or segment1.lower.y - self._event_point.y > EPSILON:
            raise ValueError(f"Event point {self._event_point} isn't in y-range of compared segment {segment1}.")
        elif self._event_point.y - segment2.upper.y > EPSILON or segment2.lower.y - self._event_point.y > EPSILON:
            raise ValueError(f"Event point {self._event_point} isn't in y-range of compared segment {segment2}.")
        elif segment1 == segment2:
            return CR.MATCH
        elif isinstance(segment1.intersection(segment2), LineSegment):
            segment1_coordinates = (segment1.upper.x, segment1.upper.y, segment1.lower.x, segment1.lower.y)
            segment2_coordinates = (segment2.upper.x, segment2.upper.y, segment2.lower.x, segment2.lower.y)
            if segment1_coordinates == segment2_coordinates:
                return CR.MATCH
            elif segment1_coordinates < segment2_coordinates:
                return CR.BEFORE
            else:
                return CR.AFTER
        elif abs(segment1.upper.y - segment1.lower.y) <= EPSILON:
            if self._compare_point_with_segment(self._event_point, segment2) is CR.BEFORE:
                return CR.BEFORE
            else:
                return CR.AFTER
        elif abs(segment2.upper.y - segment2.lower.y) <= EPSILON:
            if self._compare_point_with_segment(self._event_point, segment1) is CR.BEFORE:
                return CR.AFTER
            else:
                return CR.BEFORE
        else:
            return None

Now we can implement Plane Sweep itself.
We follow the approach from \[1, pp. 26–27\].

At first, the status structure is initialised as an empty binary tree, while the event queue is initialised as a binary tree containing all line segment endpoints as keys.
The event queue also stores a value for each key that consists of the list of line segments having this key as their upper endpoint.
That's why the event queue's type is <tt>BinaryTreeDict</tt> as opposed to the simple <tt>BinaryTree</tt> type used for the status structure, which doesn't support values.

During each step of the actual sweep, an event point is popped from the queue along with the segments having it as their upper endpoint.
A subsequent search in the status structure yields all other line segments containing the event point, which are promptly deleted from the status structure.
Next, those segments having the event point as their upper endpoint or containing it in their interior are inserted into the status structure.
This is done according to the new status structure order induced by the event point, as implemented in the comparator above.
Note that segments containing the event point in their interior are deleted and re-inserted, so their order in the status structure is reversed.
(Except for the relative order of overlapping line segments, which doesn't matter as stated above.)
Hence, there's no need for a separate swap procedure and the degenerate case of more than 2 line segments intersecting in one point is handled elegantly (see \[1, p. 26\]).

Then the segments that have become adjacent in the status structure due to the deletions and (re-)insertions are checked for intersections.
If there now aren't any line segments in the status structure that contain the event point, the left and right status neighbours of the event point are adjacent.
Otherwise, the left status neighbour is adjacent to the leftmost containing segment in the status structure, whereas the right status neighbour is adjacent to the rightmost such segment.
In both cases, an intersection point of two newly adjacent segments is added to the event queue if it hasn't already been visited by the sweep line.

Finally, the event point is reported along with all its containing line segments.
Points with at least two containing segments are intersection points.

In [8]:
def plane_sweep_lsi(segments: set[LineSegment]) -> PointSequenceDict:
    return PlaneSweepLSI(segments).sweep()

class PlaneSweepLSI:
    def __init__(self, segments: set[LineSegment]):
        self._status_structure_comparator = StatusStructureComparator()
        self._event_queue_comparator = EventQueueComparator()

        self._status_structure: BinaryTree[LineSegment] = BinaryTree(self._status_structure_comparator)
        self._event_queue: BinaryTreeDict[Point, list[LineSegment]] = BinaryTreeDict(self._event_queue_comparator)

        self._default_value_updater = lambda u_segments: [] if u_segments is None else u_segments
        for segment in segments:
            def upper_endpoint_value_updater(u_segments: Optional[list[LineSegment]]) -> list[LineSegment]:
                u_segments = self._default_value_updater(u_segments)
                u_segments.append(segment)
                return u_segments
            self._event_queue.update(segment.upper, upper_endpoint_value_updater)
            self._event_queue.update(segment.lower, self._default_value_updater)

    def sweep(self) -> PointSequenceDict:
        intersections = PointSequenceDict()
        while not self._event_queue.is_empty():
            event_point, containing_segments = self._handle_event()
            if len(containing_segments) >= 2:
                intersections.add(event_point, containing_segments)
            else:
                intersections.animate(event_point)    # Animate event_point even if it's not an intersection point.

        return intersections

    def _handle_event(self) -> tuple[Point, list[LineSegment]]:
        event_point, containing_segments = self._event_queue.pop_first()
        for segment in self._status_structure.search_matching(event_point):
            self._status_structure.delete(segment)
            containing_segments.append(segment)

        self._status_structure_comparator.set_event_point(event_point)

        def lower_endpoint_is_not_event_point(segment: LineSegment) -> bool:
            return self._event_queue_comparator.compare(segment.lower, event_point) is not CR.MATCH
        for segment in filter(lower_endpoint_is_not_event_point, containing_segments):
            self._status_structure.insert(segment)

        left_status_neighbour = self._status_structure.search_predecessor(event_point)
        containing_status_segments = self._status_structure.search_matching(event_point)
        right_status_neighbour = self._status_structure.search_successor(event_point)
        if not containing_status_segments:
            if left_status_neighbour is not None and right_status_neighbour is not None:
                self._find_new_event(left_status_neighbour, right_status_neighbour, event_point)
        else:
            if left_status_neighbour is not None:
                self._find_new_event(left_status_neighbour, containing_status_segments[0], event_point)
            if right_status_neighbour is not None:
                self._find_new_event(containing_status_segments[-1], right_status_neighbour, event_point)

        return event_point, containing_segments

    def _find_new_event(self, left_segment: LineSegment, right_segment: LineSegment, event_point: Point):
        intersection = left_segment.intersection(right_segment)
        if isinstance(intersection, Point):
            if self._event_queue_comparator.compare(intersection, event_point) is CR.AFTER:
                self._event_queue.update(intersection, self._default_value_updater)

The running time of Plane Sweep is in $O((n + i) \log(n))$ and the required space for its data structures is in $O(n + i)$, where $i$ is the number of intersection points.
Note that the full size of the output can be considerably greater than $i$ because the intersecting segments are part of the output as well (see \[1, p. 28\]).

Obviously, this algorithm performs worse the closer $i$ gets to the worst case of $n^2$ intersections, eventually losing out to Brute Force.
Another weakness of Plane Sweep is its sensitivity to robustness issues:
If the status structure becomes erroneous at any step, e.g. due to a wrong order of segments or a missed deletion of one segment, then the status invariants are violated at subsequents steps, likely resulting in a halt of execution.
Although the given implementation has a high success rate on well-behaved inputs, it's possible to find inputs that make it fail.
In such cases Brute Force could be used as a fallback method.

In [9]:
visualisation_tool.register_algorithm("Plane Sweep", plane_sweep_lsi, SweepLineMode())

The animations of Plane Sweep show how the sweep line passes the event points from top to bottom.

In [10]:
visualisation_tool.display()

__*Takeaways:*__

* The plane sweep technique is useful and conceptually simple, but implementation details can get tricky.

* A dynamic comparator is required for the status structure, and comparisons of line segments are numerically fragile.

## 4. References

\[1\] Mark de Berg, Otfried Cheong, Marc van Kreveld, and Mark Overmars. *Computational Geometry: Algorithms and Applications*, 3rd edition. 2008.

\[2\] David M. Mount. *[CMSC 754: Computational Geometry (Spring 2020)](https://www.cs.umd.edu/class/spring2020/cmsc754/Lects/cmsc754-spring2020-lects.pdf)*. 2020.